# Differential gene expression analysis in two brain regions

JuGEx performs differential gene expresssion in two different brain regions, as described in the publication

> *Sebastian Bludau, Thomas W. Mühleisen, Simon B. Eickhoff, Michael J. Hawrylycz, Sven Cichon, Katrin Amunts. Integration of transcriptomic and cytoarchitectonic data implicates a role for MAOA and TAC1 in the limbic-cortical network. 2018, Brain Structure and Function. https://doi.org/10.1007/s00429-018-1620-6*

For the gene expression data, `siibra` accesses the Allen Brain Atlas API (© 2015 Allen Institute for Brain Science. Allen Brain Atlas API. Available from: brain-map.org/api/index.html). 

### Initialize the analysis

The analysis is initialized with a `siibra` atlas object. It will check if the parcellation selected in the atlas is suitable for performing the analysis, which includes to verify that the given atlas object provides maps in the MNI ICBM 152 space. We explicitely select the Julich-Brain probabilistic cytoarchitectonic maps, and  tell the atlas to threshold the probability maps for filtering gene expressions instead of using the simplified labelled volume. 

In [ ]:
import siibra, siibra_jugex
from packaging.version import Version
assert Version(siibra.__version__) >= Version('1.0a08')
assert Version(siibra_jugex.__version__) >= Version("1.2")

In [ ]:
julichbrain = siibra.parcellations.get("julich")
jugex = siibra_jugex.DifferentialGeneExpression(julichbrain)

### Configure the experiment with brain regions and candidate genes

The analysis is configured by specifying some candidate genes of interest, and two regions of interest (ROI) specified by brain area names that the atlas object can resolve. Note that the siibra atlas class does fuzzy string matching to resolve region names, so you can try with a simple name of the regions to see if siibra interprets them.  Also, gene names can easily be looked up and autocompleted in siibra.gene_names. 


In [ ]:
candidate_regions = ["v1 right", "v2 right"]
candidate_genes = ["MAOA", "TAC1"]
jugex.add_candidate_genes(candidate_genes)
threshold=0.2
jugex.define_roi1(candidate_regions[0], maptype=siibra.MapType.STATISTICAL, threshold=threshold)
jugex.define_roi2(candidate_regions[1], maptype=siibra.MapType.STATISTICAL, threshold=threshold)

### Run the analysis

In [ ]:
result = jugex.run(permutations=1000)
print(result['p-values'])

The aggregated input parameters can be stored to disk.

In [ ]:
jugex.save('jugex_{}_{}.json'.format(
    "_".join(candidate_regions),
    "_".join(candidate_genes) ))

### Look at filtered positions of microarray samples in MNI space

Let's have a look at the sample positions that have been found in the Allen atlas. Since we configured brainscapes to prefer thresholded continuous maps for region filtering over the simplified parcellation map, we also plot the probability maps here.

In [ ]:
from nilearn import plotting

for regionname in candidate_regions:
    samples = jugex.get_samples(regionname)
    region = julichbrain.get_region(regionname)
    pmap = region.get_regional_map(
        siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC, 
        siibra.MapType.STATISTICAL
    )    
    display = plotting.plot_glass_brain(pmap.fetch(), cmap="viridis", title=region.name)
    display.add_markers([s['mnicoord'] for s in samples.values()])